In [1]:
import sys
import numpy as np
import tensorflow as tf
import os
project_home_path = '/home/fesian/AI_workspace/Projects/HRI_dataset_push_exps/'
sys.path.append(project_home_path)
project_home_path = '/home/fesian/AI_workspace/Projects/EnDeCoderClassifier/'
sys.path.append(project_home_path)
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True


/home/fesian/fesian_env3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from networks.PA_LSTM import PLSTM, DataLoader
from codes.GlobalConfig import mGlobalConfig
gc = mGlobalConfig.GlobalConfig()

Using TensorFlow backend.


In [3]:
batch_size = 1
model = PLSTM.PLSTM(batch_size=batch_size, lr=0.001, n_layers=2)

In [4]:
is_view = True
if is_view:
    traindata = np.load('/home/fesian/AI_workspace/datasets/HRI40/hri40_new_skel/x_sub/x_sub_train.npy')
    testdata = np.load('/home/fesian/AI_workspace/datasets/HRI40/hri40_new_skel/x_sub/x_sub_test.npy')

dl = DataLoader.DataLoader(skel_train=traindata, skel_test=testdata, mode='both')

In [9]:
def train_f(sess, model, datas, run_counter, train_writer):
    mats, labels, views = datas
    feed_dict = {
        model.skel_input: mats,
        model.action_gt: labels,
        model.plstm_keep_prob: 0.60
    }

    sess.run(model.adam_train_op, feed_dict)
    if run_counter % 10 == 0:
        merge = sess.run(model.loss_merged_op, feed_dict)
        train_writer.add_summary(merge, run_counter)
#         print(run_counter)

view_matrix = np.zeros((1,8))
total_matrix = np.zeros((1,8))
cm = np.zeros((40,40))
def eval_f(sess, model, datas):
    mats, labels, views = datas
    feed_dict = {
        model.skel_input: mats,
        model.plstm_keep_prob: 1.00
    }
    preds = sess.run(model.action_argmax_target, feed_dict)
    gt = int(labels[0][0])
    pred = preds[0]
    cm[pred, gt] += 1
    
   
#     for label, pred, view in zip(labels, preds, views):
#         view = int(view[0])
#         total_matrix[0, view] += 1
#         if int(pred) == int(label):
#             view_matrix[0, view] += 1
    
        

In [10]:
g_feat_dim = 25*3
g_T_size = 8
g_num_action = 40

with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    run_counter = 0

    saver = tf.train.Saver()
    train_writer = tf.summary.FileWriter(gc.get_log_path(gc.logs_edcc_path, id=20), sess.graph, flush_secs=30)
    
    is_train = False
    
    if is_train:
        for epochs in range(100):
            data_size = traindata.shape[0]
            anchor = 0
            print('-----------------------------{}------------------------------------'.format(epochs))
            for ind, each_batch in enumerate(range(data_size)):
                if anchor > data_size-batch_size:
                    break
                print(ind)

                datas = dl.get_samples(batch_size=batch_size, is_rotate=True, anchor=anchor)
                train_f(sess, model, datas, run_counter, train_writer)
                run_counter += 1
                anchor += batch_size

            saver.save(sess, gc.get_model_path(gc.models_edcc_path, comment='plstm_xusb', exp_num=0))
    else:
     
        saver.restore(sess, gc.get_model_path(gc.models_edcc_path, comment='plstm_xusb', exp_num=0))
        test_data_size = testdata.shape[0]
        anchor = 0
        for ind, each_batch in enumerate(range(test_data_size)):
            if anchor > test_data_size-batch_size:
                break
            datas = dl.get_test_sample(batch_size, anchor, is_rotate=True)
            result = eval_f(sess, model, datas)
            print('ind %d' % (anchor))
            anchor += batch_size
        model.generate_report(cm, '../reports/plstm_reports/cm_plstm_raw.xlsx')
        model.generate_report(cm/np.sum(cm, axis=0), '../reports/plstm_reports/cm_plstm_prop.xlsx')
            
#         print(view_matrix/total_matrix)
#         model.generate_report(view_matrix/total_matrix, '../reports/plstm_reports/plstm_xsub.xlsx')

INFO:tensorflow:Restoring parameters from /home/fesian/AI_workspace/Projects/EnDeCoderClassifier/models/exp_plstm_xusb/exp_0.ckpt
ind 0
ind 1
ind 2
ind 3
ind 4
ind 5
ind 6
ind 7
ind 8
ind 9
ind 10
ind 11
ind 12
ind 13
ind 14
ind 15
ind 16
ind 17
ind 18
ind 19
ind 20
ind 21
ind 22
ind 23
ind 24
ind 25
ind 26
ind 27
ind 28
ind 29
ind 30
ind 31
ind 32
ind 33
ind 34
ind 35
ind 36
ind 37
ind 38
ind 39
ind 40
ind 41
ind 42
ind 43
ind 44
ind 45
ind 46
ind 47
ind 48
ind 49
ind 50
ind 51
ind 52
ind 53
ind 54
ind 55
ind 56
ind 57
ind 58
ind 59
ind 60
ind 61
ind 62
ind 63
ind 64
ind 65
ind 66
ind 67
ind 68
ind 69
ind 70
ind 71
ind 72
ind 73
ind 74
ind 75
ind 76
ind 77
ind 78
ind 79
ind 80
ind 81
ind 82
ind 83
ind 84
ind 85
ind 86
ind 87
ind 88
ind 89
ind 90
ind 91
ind 92
ind 93
ind 94
ind 95
ind 96
ind 97
ind 98
ind 99
ind 100
ind 101
ind 102
ind 103
ind 104
ind 105
ind 106
ind 107
ind 108
ind 109
ind 110
ind 111
ind 112
ind 113
ind 114
ind 115
ind 116
ind 117
ind 118
ind 119
ind 120
ind 121
ind 

ind 1023
ind 1024
ind 1025
ind 1026
ind 1027
ind 1028
ind 1029
ind 1030
ind 1031
ind 1032
ind 1033
ind 1034
ind 1035
ind 1036
ind 1037
ind 1038
ind 1039
ind 1040
ind 1041
ind 1042
ind 1043
ind 1044
ind 1045
ind 1046
ind 1047
ind 1048
ind 1049
ind 1050
ind 1051
ind 1052
ind 1053
ind 1054
ind 1055
ind 1056
ind 1057
ind 1058
ind 1059
ind 1060
ind 1061
ind 1062
ind 1063
ind 1064
ind 1065
ind 1066
ind 1067
ind 1068
ind 1069
ind 1070
ind 1071
ind 1072
ind 1073
ind 1074
ind 1075
ind 1076
ind 1077
ind 1078
ind 1079
ind 1080
ind 1081
ind 1082
ind 1083
ind 1084
ind 1085
ind 1086
ind 1087
ind 1088
ind 1089
ind 1090
ind 1091
ind 1092
ind 1093
ind 1094
ind 1095
ind 1096
ind 1097
ind 1098
ind 1099
ind 1100
ind 1101
ind 1102
ind 1103
ind 1104
ind 1105
ind 1106
ind 1107
ind 1108
ind 1109
ind 1110
ind 1111
ind 1112
ind 1113
ind 1114
ind 1115
ind 1116
ind 1117
ind 1118
ind 1119
ind 1120
ind 1121
ind 1122
ind 1123
ind 1124
ind 1125
ind 1126
ind 1127
ind 1128
ind 1129
ind 1130
ind 1131
ind 1132
ind 1133
i

ind 1943
ind 1944
ind 1945
ind 1946
ind 1947
ind 1948
ind 1949
ind 1950
ind 1951
ind 1952
ind 1953
ind 1954
ind 1955
ind 1956
ind 1957
ind 1958
ind 1959
ind 1960
ind 1961
ind 1962
ind 1963
ind 1964
ind 1965
ind 1966
ind 1967
ind 1968
ind 1969
ind 1970
ind 1971
ind 1972
ind 1973
ind 1974
ind 1975
ind 1976
ind 1977
ind 1978
ind 1979
ind 1980
ind 1981
ind 1982
ind 1983
ind 1984
ind 1985
ind 1986
ind 1987
ind 1988
ind 1989
ind 1990
ind 1991
ind 1992
ind 1993
ind 1994
ind 1995
ind 1996
ind 1997
ind 1998
ind 1999
ind 2000
ind 2001
ind 2002
ind 2003
ind 2004
ind 2005
ind 2006
ind 2007
ind 2008
ind 2009
ind 2010
ind 2011
ind 2012
ind 2013
ind 2014
ind 2015
ind 2016
ind 2017
ind 2018
ind 2019
ind 2020
ind 2021
ind 2022
ind 2023
ind 2024
ind 2025
ind 2026
ind 2027
ind 2028
ind 2029
ind 2030
ind 2031
ind 2032
ind 2033
ind 2034
ind 2035
ind 2036
ind 2037
ind 2038
ind 2039
ind 2040
ind 2041
ind 2042
ind 2043
ind 2044
ind 2045
ind 2046
ind 2047
ind 2048
ind 2049
ind 2050
ind 2051
ind 2052
ind 2053
i

ind 2858
ind 2859
ind 2860
ind 2861
ind 2862
ind 2863
ind 2864
ind 2865
ind 2866
ind 2867
ind 2868
ind 2869
ind 2870
ind 2871
ind 2872
ind 2873
ind 2874
ind 2875
ind 2876
ind 2877
ind 2878
ind 2879
ind 2880
ind 2881
ind 2882
ind 2883
ind 2884
ind 2885
ind 2886
ind 2887
ind 2888
ind 2889
ind 2890
ind 2891
ind 2892
ind 2893
ind 2894
ind 2895
ind 2896
ind 2897
ind 2898
ind 2899
ind 2900
ind 2901
ind 2902
ind 2903
ind 2904
ind 2905
ind 2906
ind 2907
ind 2908
ind 2909
ind 2910
ind 2911
ind 2912
ind 2913
ind 2914
ind 2915
ind 2916
ind 2917
ind 2918
ind 2919
ind 2920
ind 2921
ind 2922
ind 2923
ind 2924
ind 2925
ind 2926
ind 2927
ind 2928
ind 2929
ind 2930
ind 2931
ind 2932
ind 2933
ind 2934
ind 2935
ind 2936
ind 2937
ind 2938
ind 2939
ind 2940
ind 2941
ind 2942
ind 2943
ind 2944
ind 2945
ind 2946
ind 2947
ind 2948
ind 2949
ind 2950
ind 2951
ind 2952
ind 2953
ind 2954
ind 2955
ind 2956
ind 2957
ind 2958
ind 2959
ind 2960
ind 2961
ind 2962
ind 2963
ind 2964
ind 2965
ind 2966
ind 2967
ind 2968
i

ind 3777
ind 3778
ind 3779
ind 3780
ind 3781
ind 3782
ind 3783
ind 3784
ind 3785
ind 3786
ind 3787
ind 3788
ind 3789
ind 3790
ind 3791
ind 3792
ind 3793
ind 3794
ind 3795
ind 3796
ind 3797
ind 3798
ind 3799
ind 3800
ind 3801
ind 3802
ind 3803
ind 3804
ind 3805
ind 3806
ind 3807
ind 3808
ind 3809
ind 3810
ind 3811
ind 3812
ind 3813
ind 3814
ind 3815
ind 3816
ind 3817
ind 3818
ind 3819
ind 3820
ind 3821
ind 3822
ind 3823
ind 3824
ind 3825
ind 3826
ind 3827
ind 3828
ind 3829
ind 3830
ind 3831
ind 3832
ind 3833
ind 3834
ind 3835
ind 3836
ind 3837
ind 3838
ind 3839
ind 3840
ind 3841
ind 3842
ind 3843
ind 3844
ind 3845
ind 3846
ind 3847
ind 3848
ind 3849
ind 3850
ind 3851
ind 3852
ind 3853
ind 3854
ind 3855
ind 3856
ind 3857
ind 3858
ind 3859
ind 3860
ind 3861
ind 3862
ind 3863
ind 3864
ind 3865
ind 3866
ind 3867
ind 3868
ind 3869
ind 3870
ind 3871
ind 3872
ind 3873
ind 3874
ind 3875
ind 3876
ind 3877
ind 3878
ind 3879
ind 3880
ind 3881
ind 3882
ind 3883
ind 3884
ind 3885
ind 3886
ind 3887
i

...

ind 6522
ind 6523
ind 6524
ind 6525
ind 6526
ind 6527
ind 6528
ind 6529
ind 6530
ind 6531
ind 6532
ind 6533
ind 6534
ind 6535
ind 6536
ind 6537
ind 6538
ind 6539
ind 6540
ind 6541
ind 6542
ind 6543
ind 6544
ind 6545
ind 6546
ind 6547
ind 6548
ind 6549
ind 6550
ind 6551
ind 6552
ind 6553
ind 6554
ind 6555
ind 6556
ind 6557
ind 6558
ind 6559
ind 6560
ind 6561
ind 6562
ind 6563
ind 6564
ind 6565
ind 6566
ind 6567
ind 6568
ind 6569
ind 6570
ind 6571
ind 6572
ind 6573
ind 6574
ind 6575
ind 6576
ind 6577
ind 6578
ind 6579
ind 6580
ind 6581
ind 6582
ind 6583
ind 6584
ind 6585
ind 6586
ind 6587
ind 6588
ind 6589
ind 6590
ind 6591
ind 6592
ind 6593
ind 6594
ind 6595
ind 6596
ind 6597
ind 6598
ind 6599
ind 6600
ind 6601
ind 6602
ind 6603
ind 6604
ind 6605
ind 6606
ind 6607
ind 6608
ind 6609
ind 6610
ind 6611
ind 6612
ind 6613
ind 6614
ind 6615
ind 6616
ind 6617
ind 6618
ind 6619
ind 6620
ind 6621
ind 6622
ind 6623
ind 6624
ind 6625
ind 6626
ind 6627
ind 6628
ind 6629
ind 6630
ind 6631
ind 6632
i

ind 7433
ind 7434
ind 7435
ind 7436
ind 7437
ind 7438
ind 7439
ind 7440
ind 7441
ind 7442
ind 7443
ind 7444
ind 7445
ind 7446
ind 7447
ind 7448
ind 7449
ind 7450
ind 7451
ind 7452
ind 7453
ind 7454
ind 7455
ind 7456
ind 7457
ind 7458
ind 7459
ind 7460
ind 7461
ind 7462
ind 7463
ind 7464
ind 7465
ind 7466
ind 7467
ind 7468
ind 7469
ind 7470
ind 7471
ind 7472
ind 7473
ind 7474
ind 7475
ind 7476
ind 7477
ind 7478
ind 7479
ind 7480
ind 7481
ind 7482
ind 7483
ind 7484
ind 7485
ind 7486
ind 7487
ind 7488
ind 7489
ind 7490
ind 7491
ind 7492
ind 7493
ind 7494
ind 7495
ind 7496
ind 7497
ind 7498
ind 7499
ind 7500
ind 7501
ind 7502
ind 7503
ind 7504
ind 7505
ind 7506
ind 7507
ind 7508
ind 7509
ind 7510
ind 7511
ind 7512
ind 7513
ind 7514
ind 7515
ind 7516
ind 7517
ind 7518
ind 7519
ind 7520
ind 7521
ind 7522
ind 7523
ind 7524
ind 7525
ind 7526
ind 7527
ind 7528
ind 7529
ind 7530
ind 7531
ind 7532
ind 7533
ind 7534
ind 7535
ind 7536
ind 7537
ind 7538
ind 7539
ind 7540
ind 7541
ind 7542
ind 7543
i

ind 8351
ind 8352
ind 8353
ind 8354
ind 8355
ind 8356
ind 8357
ind 8358
ind 8359
ind 8360
ind 8361
ind 8362
ind 8363
ind 8364
ind 8365
ind 8366
ind 8367
ind 8368
ind 8369
ind 8370
ind 8371
ind 8372
ind 8373
ind 8374
ind 8375
ind 8376
ind 8377
ind 8378
ind 8379
ind 8380
ind 8381
ind 8382
ind 8383
ind 8384
ind 8385
ind 8386
ind 8387
ind 8388
ind 8389
ind 8390
ind 8391
ind 8392
ind 8393
ind 8394
ind 8395
ind 8396
ind 8397
ind 8398
ind 8399
ind 8400
ind 8401
ind 8402
ind 8403
ind 8404
ind 8405
ind 8406
ind 8407
ind 8408
ind 8409
ind 8410
ind 8411
ind 8412
ind 8413
ind 8414
ind 8415
ind 8416
ind 8417
ind 8418
ind 8419
ind 8420
ind 8421
ind 8422
ind 8423
ind 8424
ind 8425
ind 8426
ind 8427
ind 8428
ind 8429
ind 8430
ind 8431
ind 8432
ind 8433
ind 8434
ind 8435
ind 8436
ind 8437
ind 8438
ind 8439
ind 8440
ind 8441
ind 8442
ind 8443
ind 8444
ind 8445
ind 8446
ind 8447
ind 8448
ind 8449
ind 8450
ind 8451
ind 8452
ind 8453
ind 8454
ind 8455
ind 8456
ind 8457
ind 8458
ind 8459
ind 8460
ind 8461
i

ind 9268
ind 9269
ind 9270
ind 9271
ind 9272
ind 9273
ind 9274
ind 9275
ind 9276
ind 9277
ind 9278
ind 9279
ind 9280
ind 9281
ind 9282
ind 9283
ind 9284
ind 9285
ind 9286
ind 9287
ind 9288
ind 9289
ind 9290
ind 9291
ind 9292
ind 9293
ind 9294
ind 9295
ind 9296
ind 9297
ind 9298
ind 9299
ind 9300
ind 9301
ind 9302
ind 9303
ind 9304
ind 9305
ind 9306
ind 9307
ind 9308
ind 9309
ind 9310
ind 9311
ind 9312
ind 9313
ind 9314
ind 9315
ind 9316
ind 9317
ind 9318
ind 9319
ind 9320
ind 9321
ind 9322
ind 9323
ind 9324
ind 9325
ind 9326
ind 9327
ind 9328
ind 9329
ind 9330
ind 9331
ind 9332
ind 9333
ind 9334
ind 9335
ind 9336
ind 9337
ind 9338
ind 9339
ind 9340
ind 9341
ind 9342
ind 9343
ind 9344
ind 9345
ind 9346
ind 9347
ind 9348
ind 9349
ind 9350
ind 9351
ind 9352
ind 9353
ind 9354
ind 9355
ind 9356
ind 9357
ind 9358
ind 9359
ind 9360
ind 9361
ind 9362
ind 9363
ind 9364
ind 9365
ind 9366
ind 9367
ind 9368
ind 9369
ind 9370
ind 9371
ind 9372
ind 9373
ind 9374
ind 9375
ind 9376
ind 9377
ind 9378
i

ind 10169
ind 10170
ind 10171
ind 10172
ind 10173
ind 10174
ind 10175
ind 10176
ind 10177
ind 10178
ind 10179
ind 10180
ind 10181
ind 10182
ind 10183
ind 10184
ind 10185
ind 10186
ind 10187
ind 10188
ind 10189
ind 10190
ind 10191
ind 10192
ind 10193
ind 10194
ind 10195
ind 10196
ind 10197
ind 10198
ind 10199
ind 10200
ind 10201
ind 10202
ind 10203
ind 10204
ind 10205
ind 10206
ind 10207
ind 10208
ind 10209
ind 10210
ind 10211
ind 10212
ind 10213
ind 10214
ind 10215
ind 10216
ind 10217
ind 10218
ind 10219
ind 10220
ind 10221
ind 10222
ind 10223
ind 10224
ind 10225
ind 10226
ind 10227
ind 10228
ind 10229
ind 10230
ind 10231
ind 10232
ind 10233
ind 10234
ind 10235
ind 10236
ind 10237
ind 10238
ind 10239
ind 10240
ind 10241
ind 10242
ind 10243
ind 10244
ind 10245
ind 10246
ind 10247
ind 10248
ind 10249
ind 10250
ind 10251
ind 10252
ind 10253
ind 10254
ind 10255
ind 10256
ind 10257
ind 10258
ind 10259
ind 10260
ind 10261
ind 10262
ind 10263
ind 10264
ind 10265
ind 10266
ind 10267
ind 10268
